# Itemgetter

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [ ]:
pip show langchain

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [ ]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

In [ ]:
chain = prompt_template | chat | StrOutputParser()

In [ ]:
chain.invoke({'message_log':'''The AI provides an interesting fact about octopuses, stating that they have three hearts. 
Two hearts pump blood to the gills, while the third heart pumps it to the rest of the body.''', 
              'question':"Can you elaborate a bit more on this fact?"})

In [ ]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')

In [ ]:
chat_memory.load_memory_variables({})

In [ ]:
RunnablePassthrough.assign(message_log = chat_memory.load_memory_variables).invoke(
    {'question':"Can you give me an interesting fact I probably didn't know about?"})

In [ ]:
RunnablePassthrough.assign(first_letter = lambda x: list(x['input'])[0], 
                           second_letter = lambda x: list(x['input'])[1]
                          ).invoke({'input':'hi'})

In [ ]:
[0, 1, 2, 3].__getitem__(1)

In [ ]:
(0, 1, 2, 3).__getitem__(1)

In [ ]:
'hi'.__getitem__(1)

In [ ]:
{'key':'value'}.__getitem__('key')

In [ ]:
RunnableLambda(itemgetter('message_log')).invoke({'message_log':''})

In [ ]:
RunnablePassthrough.assign(message_log = 
                           RunnableLambda(chat_memory.load_memory_variables) | 
                           RunnableLambda(itemgetter('message_log')) ).invoke(
    {'question':"Can you give me an interesting fact I probably didn't know about?"})